In [ ]:
import os, glob
from PIL import Image
import numpy as np
import subprocess

In [ ]:
out_dir = 'render_out'
if not os.path.exists(out_dir):
    os.mkdir(out_dir)

In [ ]:
robot_paths = glob.glob('tests/socnav/render*_robot*')

for series in glob.glob('tests/socnav/render*_bg*'):
    print(series)
    seq = series.split('/')[-1][7:-16]
    
    seq_out = os.path.join(out_dir,seq)
    if not os.path.exists(seq_out):
        os.mkdir(seq_out)
    
    sorted_img = sorted(glob.glob(os.path.join(series,seq,'*.png')))
    
    robot_idx = []
    for i in range(1,5):
        pot_series = series.replace('bg','robot{}'.format(i))
        if pot_series in robot_paths:
            robot_idx.append(i)
    most_recent = {}

    for idx,img_path in enumerate(sorted_img):
        bg = Image.open(img_path)
        to_paste = []
        valid = False
        for rob_idx in robot_idx:
            r_path = img_path.replace('_bg_','_robot{}_'.format(rob_idx))
            try:
                fg = np.array(Image.open(r_path))
                fg_v = fg.reshape((-1,4))
                fg_v[fg_v.sum(1) == 255] = 0
                valid = True
                most_recent[rob_idx] = fg
                to_paste.append(fg)
            except:
                to_paste.append(most_recent[rob_idx])
        if valid:
            for fg in to_paste:
                bg.paste(Image.fromarray(fg),(0,0),Image.fromarray(fg))
            bg.save(os.path.join(seq_out,'{:03d}.png'.format(idx)))
            #print(idx)
        else:
            break
    ffmpeg_pngs_to_mp4: str = "ffmpeg {ow} {logs} {fps} {inputs} {mp4} {q} {pad} {out}".format(
        ow="-y",  # force overwrite
        logs="-hide_banner -loglevel error",  # ignore logs
        fps="-framerate {}".format(20),  # set framerate of resulting movie
        inputs="-pattern_type glob -i '{}'".format(os.path.join(seq_out, "*.png")),
        mp4="-c:v h264 -pix_fmt yuv420p",  # video codec
        q="-preset veryfast",  # decrease the quality by 0
        pad='-vf "pad=ceil(iw/2)*2:ceil(ih/2)*2"', # make image work
        out='comp_{}.mp4'.format(seq),
    )
    error_val = subprocess.call(ffmpeg_pngs_to_mp4, shell=True)